In [1]:
%matplotlib ipympl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from windrose import WindroseAxes
import windrose
from physoce.util import compasstransform

### Load data

In [2]:
# Load data
xls_file = '../data/Rover_II_Current_Mag_Hourly_Avg_pad.xlsx'
df = pd.read_excel(xls_file,parse_dates=[6])

# Datetime index
df = df.set_index('Date_time_R')

# Create u and v components with units m/s
df['u'] = df['Easting (cm/sec)']/100
df['v'] = df['Northing (cm/sec)']/100

In [14]:
df['current_mag'] = np.sqrt(df['u']**2 + df['v']**2)
df['current_dir'] = np.arctan2(-df['v'],-df['u']) # negative for direction FROM

# Make degrees positive 
ni, = np.where(df['current_dir']<0)
df['current_dir'][ni] = df['current_dir'][ni]+2*np.pi

# convert from polar coordinates (counter-clockwise from positive x) to compass directions (clockwise from true north)
df['current_dir_compass'] = compasstransform(df['current_dir']*180/np.pi)

/Users/tomconnolly/programs/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/lib/python3.5/site-packages/physoce/util.py:87: RuntimeWarning: invalid value encountered in less
  neg = theta_out < 0


### Start and end times for calculations

Show start and end times for each deployment

Time period of interest (jellyfish): 14 October 2014 to 13 Nov 2017

Full time period: 11/20/2011 to 11/9/2017

In [16]:
t1 = np.datetime64('2014-10-16')
t2 = np.datetime64('2017-11-09')

# find data in time period
ii, = np.where((df.index >= t1) & (df.index <= t2))

plt.figure(figsize=(7.5,8))
plt.subplot(211)
plt.plot(df.index,df['u'])
# plt.xlim([t1_list[0]-np.timedelta64(3,'D'),
#           t2_list[-1]+np.timedelta64(3,'D')])
yl = plt.ylim()

plt.plot([t1,t1],yl,'g')
plt.plot([t2,t2],yl,'r')
    
plt.ylim(yl)
plt.title('eastward velocity')
plt.ylabel('[m/s]')
plt.axhline(0,linestyle='--',color='k')

plt.subplot(212)
plt.plot(df.index,df['v'])
# plt.xlim([t1_list[0]-np.timedelta64(3,'D'),
#           t2_list[-1]+np.timedelta64(3,'D')])
yl = plt.ylim()

plt.plot([t1,t1],yl,'g')
plt.plot([t2,t2],yl,'r')
    
plt.ylim(yl)
plt.title('northward velocity')
plt.ylabel('[m/s]')
plt.axhline(0,linestyle='--',color='k')

FigureCanvasNbAgg()

In [21]:
ax = WindroseAxes.from_ax()
ax.bar(df['current_dir_compass'][ii],df['current_mag'][ii], nsector=12, normed=True)
ax.legend(decimal_places=2,fontsize=8,loc='lower left')
#ax.set_legend()

FigureCanvasNbAgg()

### Progressive vector diagrams

For each deployment, two plots are shown:
* A simple trajectory plot, with the starting point
* A trajectory plot colored by time

In [6]:
# time between each sample (in seconds)
dt = np.diff(df.index)/np.timedelta64(1,'s')

# unix time
df['unix_time'] =  (df.index - pd.Timestamp('1970-01-01 00:00:00'))/np.timedelta64(1,'s')


# find data in time period
ii, = np.where((df.index >= t1) & (df.index <= t2))

# cumulative integral using trapezoidal rule
x = np.cumsum(0.5*(df['u'][ii][:-1]+df['u'][ii][1:])
              *dt[ii],axis=0)
y = np.cumsum(0.5*(df['v'][ii][:-1]+df['v'][ii][1:])
              *dt[ii],axis=0)
t = np.cumsum(0.5*(df['unix_time'][ii][:-1]+df['unix_time'][ii][1:])
              *dt[ii],axis=0)

# plot progressive vector diagram
plt.figure()
plt.plot(x/1000,y/1000)
plt.plot(0,0,'go')
plt.gca().axis('equal')
plt.xlabel('distance east [km]')
plt.ylabel('distance north [km]')
plt.title(str(t1)+' - '+str(t2))
plt.savefig('figures/progressive_vector_original_'+
        str(t1.astype(object).year)+'-'+
        str(t2.astype(object).year)+
        '.pdf')

plt.figure()
plt.scatter(x/1000,y/1000,6,t,cmap='plasma')
plt.gca().axis('equal')
plt.xlabel('distance east [km]')
plt.ylabel('distance north [km]')
plt.title(str(t1)+' - '+str(t2))

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0.5,1,'2014-10-16 - 2017-11-09')

In [26]:
plt.figure()
plt.hist2d(df['u'][ii],df['v'][ii],bins=50,cmap='gist_heat_r');
plt.xlabel('eastward velocity [m/s]')
plt.ylabel('northward velocity [m/s]')    
plt.title(str(t1)+' - '+str(t2))
plt.colorbar();
plt.axis('equal');

/Users/tomconnolly/programs/anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()